In [ ]:
# Block comment: Import Required Libraries
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Block comment: Define the Github API endpoint to search for repositories
github_api = "https://api.github.com/search/repositories"

# Define the query parameters
query_params = {
    "q": "openai chatGPT",
    "sort": "stars",
    "order": "desc"
}

In [ ]:
# Block comment: Send a GET request to the Github API to search for repositories
response = requests.get(github_api, params=query_params)

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to a JSON object
    data = response.json()

    # Extract the relevant information from the JSON object
    repositories = data["items"]
    repo_data = []
    for repo in repositories:
        repo_data.append(
            {
                "name": repo["name"],
                "full_name": repo["full_name"],
                "html_url": repo["html_url"],
                "contributors_url": repo["contributors_url"],
                "stargazers_count": repo["stargazers_count"],
                "forks_count": repo["forks_count"],
                "open_issues_count": repo["open_issues_count"],
                "contributors": []
            }
        )

    # Block comment: Loop over each repository to get the contributors
    for repo in repo_data:
        response = requests.get(repo["contributors_url"])
        if response.status_code == 200:
            contributors = response.json()
            for contributor in contributors:
                repo["contributors"].append(contributor["login"])

    # Block comment: Create a Pandas dataframe from the repository data
    df = pd.DataFrame(repo_data)

    # Create a heatmap of the contributors for each repository
    contributors_pivot = df.pivot_table(
        index="name", columns="contributors", values="contributors", aggfunc="count", fill_value=0
    )
    sns.heatmap(contributors_pivot, cmap="YlGnBu")
    plt.show()

# Block comment: If the request was unsuccessful, print an error message
else:
    print("An error occurred while trying to retrieve the repository data")